<a href="https://colab.research.google.com/github/leonardo3108/ia025a/blob/main/Aula_9_MultiheadAttention_Causal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
nome = 'Leonardo Augusto da Silva Pacheco'
print(f'Meu nome é {nome}.')

Meu nome é Leonardo Augusto da Silva Pacheco.


#  Exercício: Modelo de Linguagem com auto-atenção

Este exercício é similar ao da Aula 8, mas iremos agora treinar uma rede neural com **duas camadas** de auto-atenção **causais** para prever a próxima palavra de um texto, data as palavras anteriores como entrada. 

Iremos também trabalhar com sequencias de tamanho variável.

Na camada de auto-atenção, não se esqueça de implementar:
- Embeddings de posição
- Projeções lineares (WQ, WK, WV, WO)
- Conexões residuais
- Camada de feed forward (2-layer MLP)


O dataset usado neste exercício (BrWaC) possui um tamanho razoável e você vai precisar rodar seus experimentos com GPU.

Alguns conselhos úteis:
- **ATENÇÃO:** o dataset é bem grande. Não dê comando de imprimí-lo.
- Durante a depuração, faça seu dataset ficar bem pequeno, para que a depuração seja mais rápida e não precise de GPU. Somente ligue a GPU quando o seu laço de treinamento já está funcionando
- Não deixe para fazer esse exercício na véspera. Ele é trabalhoso.

In [2]:
# iremos utilizar a biblioteca dos transformers para ter acesso ao tokenizador do BERT.
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 5.2 MB/s 
     |████████████████████████████████| 86 kB 5.1 MB/s 
     |████████████████████████████████| 6.6 MB 30.7 MB/s 
     |████████████████████████████████| 596 kB 41.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


## Importação dos pacotes

In [3]:
import collections
import itertools
import functools
import math
import random

import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
#from tqdm import tqdm_notebook
from tqdm.notebook import tqdm

In [4]:
# Check which GPU we are using
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [5]:
if torch.cuda.is_available(): 
   dev = "cuda:0"
else: 
   dev = "cpu"
device = torch.device(dev)
print('Using {}'.format(device))

Using cpu


## Implementação do MyDataset

In [6]:
from typing import List
from torch import LongTensor
from math import ceil

def tokenize(text: str, tokenizer):
    # Recomenda-se usar o tokenizer.batch_encode_plus pois é mais rápido.
    return tokenizer(text, return_tensors=None, add_special_tokens=False).input_ids


class MyDataset():
    def __init__(self, texts: List[str], tokenizer, max_seq_length: int):
        # Escreva aqui seu código.
        self.tokenizer = tokenizer
        self.max_seq_length = max_seq_length
        self.all_tokens = []
        self.text_index = []
        self.first_ngram = []
        self.total_ngrams = 0
        self.padding = [0] * max_seq_length
        self.eos = tokenize('[SEP]', self.tokenizer)
        for i, text in enumerate(tqdm(texts, desc = 'Tokenizing')):
            tokens = tokenize('[CLS] ' + text, self.tokenizer)
            self.all_tokens.append(tokens)
            qtty = int(ceil(1.0 * len(tokens) / max_seq_length))
            #print('qtty:', qtty)
            self.text_index += [i] * qtty
            self.first_ngram.append(self.total_ngrams)
            self.total_ngrams += qtty

    def __len__(self):
        # Escreva aqui seu código.
        return self.total_ngrams

    def __getitem__(self, idx):
        # Escreva aqui seu código.
        text_index = self.text_index[idx]
        tokens = self.all_tokens[text_index]
        ngram_index = idx - self.first_ngram[text_index]
        first = self.max_seq_length * ngram_index
        after = first + self.max_seq_length
        size = len(tokens)
        if after >= size:
            x = tokens[first     :] + self.padding[: after - size]
            y = tokens[first + 1 :] + self.eos + self.padding[: after - size]
        else:
            x = tokens[first     : after]
            y = tokens[first + 1 : after + 1]
        return LongTensor(x), LongTensor(y)

In [34]:
from typing import List


def tokenize(text: str, tokenizer):
    # Recomenda-se usar o tokenizer.batch_encode_plus pois é mais rápido.
    return tokenizer(text, return_tensors=None, add_special_tokens=False).input_ids


class MyDataset():
    def __init__(self, texts: List[str], tokenizer, max_seq_length: int):
        # Escreva aqui seu código.
        self.max_seq_length = max_seq_length
        self.padding = [tokenizer.pad_token_id] * max_seq_length

        inputs = []
        targets = []
    
        for text in tqdm(texts):
            tokens = tokenize('[CLS] ' + text, tokenizer)
            sentences = len(tokens) // max_seq_length
            for i in range(sentences + 1):
                first = i * max_seq_length
                x = tokens[first : first + max_seq_length]
                y = tokens[first + 1:first + max_seq_length + 1]
                if len(x) < max_seq_length:
                    x += self.padding[:max_seq_length - len(x)]
                if len(y) < max_seq_length:
                    y += self.padding[:max_seq_length - len(y)]
                inputs.append(x)
                targets.append(y)
        
        self.inputs  = LongTensor(inputs)
        self.targets = LongTensor(targets)
        

    def __len__(self):
        # Escreva aqui seu código.
        return len(self.inputs)

    def __getitem__(self, idx):
        # Escreva aqui seu código.
        return self.inputs[idx], self.targets[idx]

## Testando se a implementação do MyDataset está correta

In [35]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")

dummy_texts = ['Eu gosto de correr', 'Ela gosta muito de comer pizza']

dummy_dataset = MyDataset(texts=dummy_texts, tokenizer=tokenizer, max_seq_length=9)
dummy_loader = DataLoader(dummy_dataset, batch_size=6, shuffle=False)
assert len(dummy_dataset) == 2
print('Passou no assert de tamanho do dataset.')

first_batch_input, first_batch_target = next(iter(dummy_loader))

correct_first_batch_input = torch.LongTensor(
    [[  101,  3396, 10303,   125, 13239,     0,     0,     0,     0],
     [  101,  1660,  5971,   785,   125,  1847, 13779, 15616,     0]])

correct_first_batch_target = torch.LongTensor(
    [[ 3396, 10303,   125, 13239,   0,     0,     0,     0,     0],
     [ 1660,  5971,   785,   125,  1847, 13779, 15616,   0,     0]])

assert torch.equal(first_batch_input, correct_first_batch_input)
assert torch.equal(first_batch_target, correct_first_batch_target)

print('Passou no assert de dataset.')

  0%|          | 0/2 [00:00<?, ?it/s]

Passou no assert de tamanho do dataset.
Passou no assert de dataset.


In [36]:
dummy_texts = ['Eu gosto de correr mas estou cansado', 'Ela gosta muito de comer pizza mas está de dieta agora']

dummy_dataset = MyDataset(texts=dummy_texts, tokenizer=tokenizer, max_seq_length=9)
dummy_loader = DataLoader(dummy_dataset, batch_size=100, shuffle=False)
print(len(dummy_dataset))
for x, y in dummy_loader:
    print('Next batch:')
    print(x)
    print(y)


  0%|          | 0/2 [00:00<?, ?it/s]

4
Next batch:
tensor([[  101,  3396, 10303,   125, 13239,   449, 12044,   822, 13550],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0],
        [  101,  1660,  5971,   785,   125,  1847, 13779, 15616,   449],
        [  698,   125, 14559,  2535,     0,     0,     0,     0,     0]])
tensor([[ 3396, 10303,   125, 13239,   449, 12044,   822, 13550,     0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0],
        [ 1660,  5971,   785,   125,  1847, 13779, 15616,   449,   698],
        [  125, 14559,  2535,     0,     0,     0,     0,     0,     0]])


# Carregamento do dataset 

Iremos usar uma pequena amostra do dataset [BrWaC](https://www.inf.ufrgs.br/pln/wiki/index.php?title=BrWaC) para treinar e avaliar nosso modelo de linguagem.

In [37]:
!wget -nc https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula9/sample-1gb.txt

File ‘sample-1gb.txt’ already there; not retrieving.



In [38]:
# Load datasets
max_seq_length = 9

train_examples = 500
valid_examples = 100
test_examples = 100

texts = open('sample-1gb.txt').readlines()

print(f'Read {len(texts)} lines.')

max_lines = train_examples + valid_examples + test_examples
print(f'Truncating to {max_lines} lines.')
texts = texts[:max_lines]  

training_texts = texts[:-(valid_examples + test_examples)]
valid_texts = texts[-(valid_examples + test_examples):-test_examples]
test_texts = texts[-test_examples:]

training_dataset = MyDataset(texts=training_texts, tokenizer=tokenizer, max_seq_length=max_seq_length)
valid_dataset = MyDataset(texts=valid_texts, tokenizer=tokenizer, max_seq_length=max_seq_length)
test_dataset = MyDataset(texts=test_texts, tokenizer=tokenizer, max_seq_length=max_seq_length)

Read 250000 lines.
Truncating to 700 lines.


  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [39]:
print(f'training examples: {len(training_dataset)}')
print(f'valid examples: {len(valid_dataset)}')
print(f'test examples: {len(test_dataset)}')

training examples: 76294
valid examples: 10157
test examples: 9593


In [40]:
from torch.nn.modules import linear
from torch.nn import Embedding, Sequential, Linear, Flatten, ReLU, Softmax, LayerNorm
from torch import arange, div, matmul, transpose, bmm

class LanguageModel(torch.nn.Module):

    def __init__(self, vocab_size: int, max_seq_length: int, dim: int, n_layers: int, pad_token_id: int):
        """
        Implements the Self-attention, decoder-only."

        Args:
            vocab_size (int): Size of the input vocabulary.
            max_seq_length (int): Size of the sequence to consider as context for prediction.
            dim (int): Dimension of the embedding layer for each word in the context.
            n_layers (int): number of self-attention layers.
            pad_token_id (int): id of the pad token that will be ignored in the attention.
        """
        # Escreva seu código aqui.
        super(LanguageModel, self).__init__()
        self.context_size = max_seq_length
        self.embedding_dim = dim
        self.vocab_size = vocab_size
        hidden_size = 1024

        self.W_c = Embedding(vocab_size, dim)
        self.W_p = Embedding(max_seq_length, dim) 
        self.W_q = Linear(dim, dim, bias=False)
        self.W_k = Linear(dim, dim, bias=False)
        self.W_v = Linear(dim, dim, bias=False)
        self.W_o = Linear(dim, dim, bias=False)
        

        self.dense = Sequential(
            Flatten(),
            LayerNorm(dim),
            Linear(dim, hidden_size),
            ReLU(),
            Linear(hidden_size, vocab_size))

        self.softmax = Softmax(dim=-1)


    def attention(self, X, debug = False):
        Q = self.W_q(X)
        K = self.W_k(X)
        V = self.W_v(X)
        O = self.W_o(X)
        if debug: print(f"Q: {Q.shape}")
        if debug: print(f"K: {K.shape}")
        if debug: print(f"V: {V.shape}")
        if debug: print(f"O: {O.shape}")
        if debug: print()

        scores = bmm(Q, transpose(K, 1, -1))
        probs = self.softmax(scores)
        E = bmm(probs, V)
        E *= O

        if debug: print(f"scores: {scores.shape}")
        if debug: print(f"probs: {probs.shape}")
        if debug: print(f"E: {E.shape}")

        return E

    def forward(self, inputs, debug = False):
        """
        Args:
            inputs is a LongTensor of shape (batch_size, max_seq_length)
            
        Returns:
            logits of shape (batch_size, max_seq_length, vocab_size)
        """
        # Escreva seu código aqui.
        if debug: print(f"batch size    (B): {inputs.shape[0]}")
        if debug: print(f"context size  (L): {self.context_size}")
        if debug: print(f"embedding dim (D): {self.embedding_dim}")
        if debug: print(f"vocab size    (V): {self.vocab_size}")
        if debug: print()

        positions = arange(self.context_size, device=inputs.device).unsqueeze(0)
        if debug: print(f"inputs: {inputs.shape}")
        if debug: print(f"positions: {positions.shape}")

        C = self.W_c(inputs)
        P = self.W_p(positions)
        X = C + P
        if debug: print(f"C: {C.shape}")
        if debug: print(f"P: {P.shape}")
        if debug: print(f"X: {X.shape}")

        E = self.attention(X, debug)

        E += X   # residual

        last = E[:,-1,:]
        if debug: print(f"last: {last.shape}")
        logits = self.dense(last)
        return logits

## Teste o modelo com um exemplo

In [41]:
model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    max_seq_length=max_seq_length,
    dim=64,
    n_layers=2,
    pad_token_id=tokenizer.pad_token_id,
).to(device)

sample_input, _ = next(iter(DataLoader(training_dataset)))
sample_input = sample_input.to(device)
sample_output = model(sample_input)
print(f'sample_input.shape: {sample_input.shape}')
print(f'sample_output.shape: {sample_output.shape}')

sample_input.shape: torch.Size([1, 9])
sample_output.shape: torch.Size([1, 29794])


In [42]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Number of model parameters: {num_params}')

Number of model parameters: 32529314


## Assert da Perplexidade


In [45]:
random.seed(123)
np.random.seed(123)
torch.manual_seed(123)


def perplexity(logits, target, ignore_token_id: int):
    """
    Computes the perplexity.

    Args:
        logits: a FloatTensor of shape (batch_size, seq_length, vocab_size)
        target: a LongTensor of shape (batch_size, seq_length)

    Returns:
        A float corresponding to the perplexity
    """
    logits = logits.reshape(-1, logits.shape[-1])
    target = target.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target, reduction='mean', ignore_index=ignore_token_id)
    return torch.exp(loss)


n_examples = 1000

train_input_ids, train_target_ids = next(iter(DataLoader(training_dataset, batch_size=n_examples)))
train_input_ids = train_input_ids.to(device)
train_target_ids = train_target_ids.to(device)

logits = model(train_input_ids)
print(train_input_ids.shape, train_target_ids.shape, logits.shape)

my_perplexity = perplexity(logits=logits, target=train_target_ids, ignore_token_id=tokenizer.pad_token_id)

print(f'my perplexity:              {int(my_perplexity)}')
print(f'correct initial perplexity: {tokenizer.vocab_size}')

assert math.isclose(my_perplexity, tokenizer.vocab_size, abs_tol=7000)
print('Passou o no assert da perplexidade')

torch.Size([1000, 9]) torch.Size([1000, 9]) torch.Size([1000, 29794])


ValueError: ignored

## Laço de Treinamento e Validação

In [ ]:
max_examples = 150_000_000
eval_every_steps = 10000
lr = 3e-4


model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    max_seq_length=max_seq_length,
    dim=64,
    n_layers=2,
    pad_token_id=tokenizer.pad_token_id,
).to(device)

train_loader = DataLoader(training_dataset, batch_size=64, shuffle=True, drop_last=True)
validation_loader = DataLoader(valid_dataset, batch_size=64)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)


def train_step(input_ids, target_ids):
    model.train()
    model.zero_grad()
    logits = model(input_ids)
    logits = logits.reshape(-1, logits.shape[-1])
    target_ids = target_ids.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target_ids, ignore_index=model.pad_token_id)
    loss.backward()
    optimizer.step()

    return loss.item()


def validation_step(input_ids, target_ids):
    model.eval()
    logits = model(input_ids)
    logits = logits.reshape(-1, logits.shape[-1])
    target_ids = target_ids.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target_ids, ignore_index=model.pad_token_id)
    return loss.item()


train_losses = []
n_examples = 0
step = 0
while n_examples < max_examples:
    for train_input_ids, train_target_ids in train_loader:
        loss = train_step(train_input_ids.to(device), train_target_ids.to(device)) 
        train_losses.append(loss)
        
        if step % eval_every_steps == 0:
            train_ppl = np.exp(np.average(train_losses))

            with torch.no_grad():
                valid_ppl = np.exp(np.average([
                    validation_step(val_input_ids.to(device), val_target_ids.to(device))
                    for val_input_ids, val_target_ids in validation_loader]))

            print(f'{step} steps; {n_examples} examples so far; train ppl: {train_ppl:.2f}, valid ppl: {valid_ppl:.2f}')
            train_losses = []

        n_examples += len(train_input_ids)  # Increment of batch size
        step += 1
        if n_examples >= max_examples:
            break

## Avaliação final no dataset de teste


Bonus: o modelo com menor perplexidade no dataset de testes ganhará 0.5 ponto na nota final.

In [ ]:
test_loader = DataLoader(test_dataset, batch_size=64)

with torch.no_grad():
    test_ppl = np.exp(np.average([
        validation_step(test_input_ids.to(device), test_target_ids.to(device))
        for test_input_ids, test_target_ids in test_loader
    ]))

print(f'test perplexity: {test_ppl}')

## Teste seu modelo com uma sentença

Escolha uma sentença gerada pelo modelo que ache interessante.

In [ ]:
prompt = 'Eu gosto de comer pizza pois me faz'
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-max_seq_length:]  # Usamos apenas os últimos <max_seq_length> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    logits = logits[:, -1, :]  # Usamos apenas o ultimo token da sequencia
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

## Bonus 1
Quem conseguir a menor perplexidade no dataset de testes ganha 0.5 ponto na média final.

## Bonus 2
Qual é a complexidade (em notação O-grande) da função de geração de texto acima?

Quem responder corretamente a pergunta acima e deixar a função com menor complexidade ganha 0.5 ponto na média final.